In [1]:
import numpy as np
import torch
import torch.nn as nn

best_model, char_to_int = torch.load("model_epoch_28.pt")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())

# Reload the model
class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=2, batch_first=True, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(self.dropout(x))
        return x
    
model = CharModel()
model.load_state_dict(best_model)

# Randomly generate a prompt
# filename = "wonderland.txt"
# seq_length = 100
# raw_text = open(filename, 'r', encoding='utf-8').read()
# raw_text = raw_text.lower()
# start = np.random.randint(0, len(raw_text)-seq_length)
# prompt = raw_text[start:start+seq_length]
# pattern = [char_to_int[c] for c in prompt]

prompt = "THE"
pattern = [char_to_int[c] for c in prompt]


model.eval()
print('Prompt: "%s"' % prompt)
with torch.no_grad():
    i = 0
    result = ''
    while i < 200 and result != '+':
        # Format input array of int into PyTorch tensor
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)
        
        # Generate logits as output from the model
        prediction = model(x)
        
        # Convert logits into one character
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end="")
        
        # Append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
        
        i += 1
        
print()
print("Done.")


ValueError: too many values to unpack (expected 2)

In [2]:
torch.load("model_epoch_28.pt")


{'epoch': 28,
 'model_state_dict': OrderedDict([('lstm.weight_ih_l0',
               tensor([[ 0.2564],
                       [ 0.8601],
                       [-0.0143],
                       ...,
                       [ 1.0378],
                       [ 0.4845],
                       [ 0.1568]])),
              ('lstm.weight_hh_l0',
               tensor([[ 0.2296,  0.1262,  0.2682,  ..., -0.0807, -1.3929,  0.5692],
                       [ 0.1825, -0.0471,  0.0799,  ...,  0.2764,  0.2877,  0.0288],
                       [-0.0708,  0.3890,  0.3725,  ...,  0.2305,  0.4200, -0.5389],
                       ...,
                       [-0.7526,  0.4896, -0.2463,  ..., -0.0765, -0.2039,  0.2205],
                       [-0.8992,  0.2343,  0.7774,  ...,  0.5868,  0.1877,  0.1049],
                       [-0.8654, -0.1036,  0.6906,  ...,  0.5744,  0.4880, -0.2840]])),
              ('lstm.bias_ih_l0',
               tensor([-0.5965, -0.0657, -0.0908,  ..., -0.5029, -0.6030, -0.4084]))